In [2]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def prepare_dataframe(folder_dir):
    res = []
    for root, dirs, files in os.walk(folder_dir, topdown=True):
        if len(files) > 0:
            res.extend(list(zip([root]*len(files), files)))

    df = pd.DataFrame(res, columns=['Path', 'File_Name'])

    df = df[df['File_Name'] != 'Thumbs.db']
    df['ClientId'] = df.Path.apply(lambda x: int(x.split("\\")[-1]))
    df = df[df['ClientId'] < 10000]

    df['Full_Path'] = df["Path"] + '\\' + df["File_Name"]
    df['Cat'] = df.File_Name.apply(lambda x: x.split(".")[0].split("_")[-1])

    df = df[df['Cat'].map(df['Cat'].value_counts()) > 150]

    return df

In [3]:
from transferlearning import ConvBaseSearch

pretrained_models = ["vgg16", "vgg19"] #Models that should be tested
folder_dir = 'C:\\Users\\Michael\\Feels Like Home\\Francisco Cruz - Fotos_DL\\011_Fotos\\'
df = prepare_dataframe(folder_dir)
target_col = "Cat"
path_col = "Full_Path"

Using TensorFlow backend.


In [4]:
# Initialize ConvBaseSearch and pass the model names you want to test
cbs = ConvBaseSearch(pretrained_models)

In [5]:
# Fit the data to the pretrained models. Only use sample_size proportion of whole dataset.
cbs.fit(df, path_col, target_col, sample_size=0.01, data_augmentation=False)

Fitting  vgg16
Found 111 validated image filenames belonging to 12 classes.
Successfully extraced features from  vgg16
Epoch 1/1
23/23 [==============================] - 0s 4ms/step
Score:  0.30434781312942505
Fitting  vgg19
Found 111 validated image filenames belonging to 12 classes.
Successfully extraced features from  vgg19
Epoch 1/1
23/23 [==============================] - 0s 2ms/step
Score:  0.1304347813129425


In [6]:
# Get the best score 
cbs.best_score

0.30434781312942505

In [9]:
# Get the best model
best_base = cbs.best_base
print("Name:", best_base.name)
print("Keras model:", best_base.model)

Name: vgg16
Keras model: <keras.engine.training.Model object at 0x000001BF4D976BC8>
